# Building an ETL Pipeline

As the second part of the predict for Gather, you will need to build a pipeline of functions in python which does the following:

1. Function to connect to twitter and scrapes "Eskom_SA" tweets.
<br>
<br>
2. Cleans/Processes the tweets from the scraped tweets which will create a dataframe with two new columns using the following functions: <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; a) Hashtag Remover from Analyse Functions
<br>
<br>
3. Functions which connects to your SQL database and uploads the tweets into the table you store the tweets in the database.

In [4]:
# General:
import tweepy           # To consume Twitter's API
import pandas as pd     # To handle data
import numpy as np      # For numerical computation
import json
# For plotting and visualization:
from IPython.display import display
import pyodbc


# Consumer and Access details

Fill in your Consumer and Access details you should have recieved when applying for a Twitter API. 

In [5]:
# Consumer:
CONSUMER_KEY    = 'hkeVlDanGNTN8DRQF8HqlTWBa'
CONSUMER_SECRET = 'MQKqkhGXl4ukGUcQCSHxd7GfHOIjMvgABHJ0N2TdBV1nochloi'

# Access:
ACCESS_TOKEN  = '1399297879-NLwurAjgf7grsIx3AqLK3z0HaMKOpqWEjvZj7PV'
ACCESS_SECRET = 'qRuUzew94mze9sY4HpK4SS6vCVwZJW6ZjHOS2o5HjjGph'

In [6]:
# API's setup:
def twitter_setup():
    """
    Utility function to setup the Twitter's API
    with access and consumer keys from Twitter.
    """

    # Authentication and access using keys:
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

    # Return API with authentication:
    api = tweepy.API(auth, timeout=1000)
    return api

In [7]:
api = twitter_setup()
user = api.get_user("Eskom_SA")

print("User details:")
print(user.name)
print(user.description)
print(user.location)

print("Last 20 Followers:")
for follower in user.followers():
    print(follower.name)

User details:
Eskom Hld SOC Ltd

Gauteng, South African   
Last 20 Followers:
Sir Mattb
Albertina Siaga
Willie Modishe
Bongiwe Rose
Prince Macheke
Mpho Douglas
Braam Cronje
Janine. Beneke
TK
Phumzile Mdluli
Chante Du Toit
Ringetani Lucius Cha
Mo'Afrika
Refiloe
Palesa botipe
Nako
@juliekuan
Bhekzin Legend
Divah💕🌸🌹
THABANG


In [8]:
for tweet in api.search(q="@Eskom_SA", lang="en", rpp=1):
    print(tweet)

Status(_api=<tweepy.api.API object at 0x000001CEB199D550>, _json={'created_at': 'Thu Mar 05 05:53:42 +0000 2020', 'id': 1235443348380491776, 'id_str': '1235443348380491776', 'text': 'RT @firstcitizenno1: The fight is heating up. You must pay close attention here. Gov refuses to address the real problems at @Eskom_SA, but…', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'firstcitizenno1', 'name': 'The Sentinel', 'id': 3299233240, 'id_str': '3299233240', 'indices': [3, 19]}, {'screen_name': 'Eskom_SA', 'name': 'Eskom Hld SOC Ltd', 'id': 466420346, 'id_str': '466420346', 'indices': [125, 134]}], 'urls': []}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'

In [34]:
api = twitter_setup()
tweets = []
dates = []
result = pd.DataFrame()
for tweet in api.search(q="@Eskom_SA -filter:retweets", lang="en", rpp=100, count=40):
    tweets = tweets + [f"{tweet.text}"]
    dates = dates + [f"{tweet.created_at}"]
result['Tweets'] = tweets
result['Date'] = dates
result

,Tweets,Date
0,"@Eskom_SA No loadshedding nxa tsek,you know fo...",2020-03-05 06:24:42
1,@Eskom_SA link doesn’t work!!,2020-03-05 06:18:32
2,@Eskom_SA Please keep the same energy even in ...,2020-03-05 06:17:46
3,"@Eskom_SA Wow, is it going to be 2 weekends wi...",2020-03-05 06:15:33
4,@Eskom_SA I never thought we would get to a da...,2020-03-05 06:12:54
5,@AfrikPinocchio @koko_matshela @James19799836 ...,2020-03-05 06:12:09
6,@Rowan40040602 @herbert_gomba @zinwawater @Zet...,2020-03-05 06:09:42
7,@CityTshwane We woke up without electricity th...,2020-03-05 06:06:10
8,"In-depth article, Marianne Green, @dailymaveri...",2020-03-05 06:04:22
9,"Sandile Zungu on @SAfmRadio ,chairman of @Blac...",2020-03-05 06:02:30


In [35]:
result['Tweets'][0]

'@Eskom_SA No loadshedding nxa tsek,you know for sure not everyone.'

# Function 1:

Write a function which:
- Scrapes _"Eskom_SA"_ tweets from Twitter. 

Function Specifications:
- The function should return a dataframe with the scraped tweets with just the "_Tweets_" and "_Date_". 
- Will take in the ```consumer key,  consumer secret code, access token``` and ```access secret code```.

NOTE:
The dataframe should have the same column names as those in your SQL Database table where you store the tweets.

In [ ]:
def twitter_df(CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_SECRET ):

    # Code Here
    
    return None

# Function 2: Removing hashtags and the municipalities

Write a function which:
- Uses the function you wrote in the Analyse section to extract the hashtags and municipalities into it's own column in a new data frame. 

Function Specifications:
- The function should take in the pandas dataframe you created in Function 1 and return a new pandas dataframe. 

In [ ]:
twitter_df(CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_SECRET )

In [36]:
def extract_municipality_hashtags(df):
    
    """Returns a modified dataframe with two new columns appended, "municipality" and "hashtags". Information is extracted from
    twitter data that includes the municipality and the list of hashtags referred to in each tweet, respectively.
    Input must contain a column named "Tweets".
    
    Parameters
    ----------
    df: dataframe
    
    Returns
    -------
    df_new: modified dataframe 
    """
    
    mun_dict = {'@CityofCTAlerts' : 'Cape Town',
                '@CityPowerJhb' : 'Johannesburg',
                '@eThekwiniM' : 'eThekwini' ,
                '@EMMInfo' : 'Ekurhuleni',
                '@centlecutility' : 'Mangaung',
                '@NMBmunicipality' : 'Nelson Mandela Bay',
                '@CityTshwane' : 'Tshwane'}
    
    if type(df) == type(pd.DataFrame()):
        municipality = []
        data = df
        for i in data["Tweets"]:
            data_str = i.replace(":", "") # Remove ":" from the end of municipality keys and hashtags
            data_str = str.split(data_str) # Splits a sentence/multi-word string by white space into a list
            data_muni = [a for a in data_str if a[0] == "@"] # Add words containing the hashtag to new list
            municipality = municipality + [data_muni]
        for j in range(len(municipality)):
            municipality[j] = [i.replace(i, mun_dict[i]) for i in municipality[j] if i in mun_dict]
        for x in range(len(municipality)):
            if municipality[x] == []:
                municipality[x] = (np.nan)

        df_muni = pd.DataFrame({"municipality": municipality})
        df = df.join(df_muni)
    
        data_subset = df
        hashtags = []
        for j, k in data_subset.iterrows(): # Iterate over pd df
            data_subset_str = data_subset.iloc[j,0]
            data_subset_str = str.split(data_subset_str) # Splits a sentence/multi-word string by white space into a list
            data_subset_hashtags = [a for a in data_subset_str if a[0] == "#"] # Add words containing the hashtag to new list
            data_subset_hashtags = list(map(lambda b: str.lower(b), data_subset_hashtags)) # Convert all hashtags in list to lower case
            if data_subset_hashtags == []:
                data_subset_hashtags = (np.nan) # Use () instead of [], resulting nan must not have square brackets in solution
            hashtags = hashtags + [data_subset_hashtags]

        df = data_subset
        df2 = pd.DataFrame({"hashtags": hashtags})
        df = df.join(df2)
        df_new = df
    
    else:
        print("Error: input must be a data frame.")
    return df_new

In [37]:
twitter_df = extract_municipality_hashtags(result)

In [38]:
twitter_df.head()

,Tweets,Date,municipality,hashtags
0,"@Eskom_SA No loadshedding nxa tsek,you know fo...",2020-03-05 06:24:42,NaN,NaN
1,@Eskom_SA link doesn’t work!!,2020-03-05 06:18:32,NaN,NaN
2,@Eskom_SA Please keep the same energy even in ...,2020-03-05 06:17:46,NaN,NaN
3,"@Eskom_SA Wow, is it going to be 2 weekends wi...",2020-03-05 06:15:33,NaN,[#loadshedding!!]
4,@Eskom_SA I never thought we would get to a da...,2020-03-05 06:12:54,NaN,NaN


# Function 3: Updating SQL Database with pyODBC

Write a function which:
- Connects and updates your SQL database. 

Function Specifications:
- The function should take in a pandas dataframe created in Function 2. 
- Connect to your SQL database.
- Update the table you store your tweets in.
- Not return any output.

In [39]:
conn = pyodbc.connect(driver='{SQL Server}',
                      host='LAPTOP-7FR129DV\SQLEXPRESS',
                      database='gather_eskom',
                      trusted_connection='tcon',
                      user='sa',
                      autocommit=True)

In [60]:
def pyodbc_twitter(connection, df, twitter_table):
    """Extracts a dataframe of tweets and connects and updates the data in your local SQL database

    Parameters:
    -----------
    connection: SQL connection settings

    df: DataFrame of tweets with their timestamp

    twitter_table: An already existing twitter SQL database 

    Examples:
    --------
    >>> conn = pyodbc.connect(driver='{SQL Server}',
                      host='your_server_name',
                      database='your_database_name', 
                      trusted_connection='tcon',
                      user='your_user_name',
                      autocommit=True)
    
    >>> df = 
    """
    cursor = connection.cursor()
    
    for i in range(len(df.index)):
        tweet_text = df.iloc[i]['Tweets']
        tweet_date = df.iloc[i]['Date']
        tweet_municipality = df.iloc[i]['municipality']
        tweet_hashtag = df.iloc[i]['hashtags']
        cursor.execute(
            f"""
            INSERT INTO {twitter_table}
            VALUES ('{tweet_text}', {tweet_date}, '{tweet_municipality}', '{tweet_hashtag}')
            """
        )

    return None 

In [62]:
pyodbc_twitter(conn, twitter_df, tweets)

ProgrammingError: ('42000', "[42000] [Microsoft][ODBC SQL Server Driver][SQL Server]The identifier that starts with ''@Eskom_SA No loadshedding nxa tsek,you know for sure not everyone.', '@Eskom_SA link doesn’t work!!', '@Eskom_SA Please keep th' is too long. Maximum length is 128. (103) (SQLExecDirectW); [42000] [Microsoft][ODBC SQL Server Driver][SQL Server]Incorrect syntax near '06'. (102)")